In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os

In [2]:
weather = pd.read_csv('kyoto_weather.csv')
weather.head()

,Date,Temperature(C),Max_temp,Min_temp,Dew_point,Max_dew_point,Min_dew_point,Pressure,Humidity,Wind,Clouds,Description
0,2017-01-01 00:00:00,6,7,4,3.6,4.6,1.6,1028,88,2,75,Clouds
1,2017-01-01 01:00:00,7,9,5,4.6,6.6,2.6,1028,88,1,75,Clouds
2,2017-01-01 02:00:00,9,10,7,2.0,3.0,0.0,1028,65,1,40,Clouds
3,2017-01-01 03:00:00,11,11,10,0.2,0.2,-0.8,1027,46,1,20,Clouds
4,2017-01-01 04:00:00,12,12,11,0.6,0.6,-0.4,1026,43,1,20,Clouds


In [3]:
# Assign X (data) and y (target)
predictors = ['Temperature(C)','Dew_point','Pressure','Wind','Clouds']
X = weather[predictors]
y = weather["Description"]

In [4]:
# Split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
# Create the SVC Model
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [6]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [7]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] C=1, gamma=0.0001 ...............................................


/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[CV] ...... C=1, gamma=0.0001, score=0.8306211322484529, total=   3.4s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.7s remaining:    0.0s


[CV] ....... C=1, gamma=0.0001, score=0.832416322787712, total=   3.4s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.4s remaining:    0.0s


[CV] ...... C=1, gamma=0.0001, score=0.8338306173972917, total=   3.3s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.8306211322484529, total=   3.3s
[CV] C=1, gamma=0.001 ................................................
[CV] ........ C=1, gamma=0.001, score=0.832416322787712, total=   3.4s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.8338306173972917, total=   3.2s
[CV] C=1, gamma=0.01 .................................................
[CV] ........ C=1, gamma=0.01, score=0.8306211322484529, total=   3.5s
[CV] C=1, gamma=0.01 .................................................
[CV] ......... C=1, gamma=0.01, score=0.832416322787712, total=   3.4s
[CV] C=1, gamma=0.01 .................................................
[CV] ........ C=1, gamma=0.01, score=0.8338306173972917, total=   3.3s
[CV] C=5, gamma=0.0001 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [8]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 10, 'gamma': 0.0001}


In [9]:
# List the best score
print(grid.best_score_)

0.8329001681700046


In [10]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   ['Clouds' 'Clear' 'Clear' 'Clouds' 'Clouds' 'Clouds' 'Clouds' 'Clear'
 'Clouds' 'Clouds']
First 10 Actual labels: ['Clouds', 'Clear', 'Clear', 'Clouds', 'Clouds', 'Clouds', 'Clouds', 'Clear', 'Clouds', 'Clouds']


In [11]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=predictors))

                precision    recall  f1-score   support

Temperature(C)       0.99      1.00      0.99       758
     Dew_point       0.81      0.93      0.86      2598
      Pressure       0.00      0.00      0.00         1
          Wind       0.00      0.00      0.00         1
        Clouds       0.00      0.00      0.00        20

   avg / total       0.80      0.82      0.80      4361



/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 5
  .format(len(labels), len(target_names))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,Clouds,Clouds
1,Clear,Clear
2,Clear,Clear
3,Clouds,Clouds
4,Clouds,Clouds
5,Clouds,Clouds
6,Clouds,Clouds
7,Clear,Clear
8,Clouds,Clouds
9,Clouds,Clouds
